In [ ]:
from controller_modules.create_user_setting_file import CreateUserSetting
from controller_modules.log_output import LogOutput
from controller_modules.snap_graph_processing import SnapGraphProcessing
from controller_modules.specific_snap_graph_processing import SpecificSnapGraphProcessing
from controller_modules.create_input_output import CreateInputOutput
from controller_modules.pyFunc_queries import PyFuncQueries
from controller_modules.geo_position import GeoPosition

import time
import os

In [ ]:
%reload_ext autoreload
%autoreload 2

///////////////////////////This Notebook is to perform test for a sequences of snap graph operations.\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

///////////////////////////This can be done for single scenes or list of scenes created over test.geojson AOI.\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [ ]:
# Here all modules are created and initialized as well as the neccessary snap xml graph operator parameters set.

userSettings = CreateUserSetting()
logOutput = LogOutput()
snapGraphProcessing = SnapGraphProcessing()

# All snap xml graph files lie in the snap_graph_files folder of the repository.
specificSnapGraphProcessing = SpecificSnapGraphProcessing(os.getcwd() + "/snap_graph_files/", userSettings.dataPath)
specificSnapGraphProcessing.setAllProcessingParameters()

Ensure user_settings.xml file was created and all paths added in correct xml format.

//////////////////////////This is where the test processing chain starts//////////////////////////////////

In [ ]:
# Here test file is pointed to in user settings.
# This can be changed to other geojson file with different AOI.
# Size of AOI is limited to max the size of one Scene (appromimately 200km²)!!!
geojson = os.getcwd() + '/test_files/test.geojson'
userSettings.setAttribute("aoiLocation", geojson)

# Start and end date can be changed. Date Format must be: YYYY-MM-DD!!!
userSettings.setAttribute("calculationStartDate", "2021-03-01")
userSettings.setAttribute("calculationEndDate", "2021-03-10")

//////////////////////////This is to create list of scenes for batch processing//////////////////////////////////

In [ ]:
# Both product types are used to create all lists
productTypeSlc = 'SLC'
productTypeGrd = 'GRD'

# Internally wkt is used by Snap for calculations 
wktAoiSettings = GeoPosition().loadWktFromGeojson(geojson)

# All tiles underlying Aoi are derived for time range from Code-De over url query.
tilesSlc = PyFuncQueries().buildSentinel1QueryTileList(geojson, userSettings.calculationStartDate, userSettings.calculationEndDate, productTypeSlc)
tilesGrd = PyFuncQueries().buildSentinel1QueryTileList(geojson, userSettings.calculationStartDate, userSettings.calculationEndDate, productTypeGrd)

# Filtered and reformatted lists are created with all duplicates removed. 
# Pairs are generated for cases where slice must be performed before processing.
# Pairs of pairs are created for coherence calculation.
# A txt file containing these lists is saved in a precreated in_output_file_list folder in the user settings main output path.
filePath = userSettings.dataPath + "in_output_file_list/"
inOutputListSlc = CreateInputOutput().generateFileList(tilesSlc, productTypeSlc, userSettings.calculationStartDate, userSettings.calculationEndDate, 
                                                       filePath)
inOutputListGrd = CreateInputOutput().generateFileList(tilesGrd, productTypeGrd, userSettings.calculationStartDate, userSettings.calculationEndDate, 
                                                       filePath)

vegIdProcessingList = inOutputListSlc[0] if len(inOutputListSlc) >= 1 else []
cohProcessingList12Days = inOutputListSlc[1] if len(inOutputListSlc) >= 2 else []
cohProcessingList6Days = inOutputListSlc[2] if len(inOutputListSlc) == 3 else []
backscatterProcessingList = inOutputListGrd

//////////////////////////Test DBackscatter calculation on single scene and Batch of scenes.//////////////////////////////////

In [ ]:
%%time
# ###################This is for calculation of backscatter ######################

# This is for testing one single list entry and diverse aoi with all possible overlapping szenarios
testElementSlice = ["/codede/Sentinel-1/SAR/GRD/2021/10/28/S1B_IW_GRDH_1SDV_20211028T172433_20211028T172458_029339_038065_C12A.SAFE, /codede/Sentinel-1/SAR/GRD/2021/10/28/S1B_IW_GRDH_1SDV_20211028T172408_20211028T172433_029339_038065_94DA.SAFE",
                    "S1B_20211028_VVVH_88_desc_BS.tif"]

testElementSingle = ["/codede/Sentinel-1/SAR/GRD/2021/03/01/S1A_IW_GRDH_1SDV_20210301T173254_20210301T173319_036808_0453E1_C7B4.SAFE","S1A_20210301_VVVH_161_desc_BS.tif"]

# Log file will be written and placed in folder in user settings main path
# Try both test elements and check for calculation times.
logOutput.createLogOutputFile("test_backscatter")
specificSnapGraphProcessing.multiSceneProcBackscatter(testElementSlice, "Test_AOI", wktAoiSettings, userSettings.backscatterOutputPath, logOutput)
logOutput.closeCurrentFile()

# This is for batch process of list
BatchProcessing(userSettings).calculateBackscatter(backscatterProcessingList, wktAoiSettings, userSettings)

//////////////////////////Test Dual pol and Compact Pol calculation on single scene and Batch of scenes.//////////////////////////////////

In [ ]:
%%time
# This is for testing one single list entry and diverse aoi with all possible overlapping szenarios
testElementSlice = ["/codede/Sentinel-1/SAR/SLC/2021/10/16/S1B_IW_SLC__1SDV_20211016T172431_20211016T172459_029164_037B03_CC7D.SAFE, /codede/Sentinel-1/SAR/SLC/2021/10/16/S1B_IW_SLC__1SDV_20211016T172406_20211016T172434_029164_037B03_3CF0.SAFE",
  "S1B_20211028_VVVH_88_desc_polVI2.tif"]

testElementSingle = ["/codede/Sentinel-1/SAR/GRD/2021/10/28/S1B_IW_GRDH_1SDV_20211028T172433_20211028T172458_029339_038065_C12A.SAFE ", "S1B_20211028_VVVH_88_desc_polVI.tif"]

# Log file will be written and placed in folder in user settings main path.
# Try both test elements and check for calculation times.
logOutput.createLogOutputFile("test_rad_veg_id")
specificSnapGraphProcessing.multiSceneProcRadVegId(testElementSlice, "Test_AOI", wktAoiSettings, userSettings.dpVegIndexPath, logOutput)
logOutput.closeCurrentFile()

# This is for batch process of list
BatchProcessing(userSettings).calculateVegIndex(vegIdProcessingList, wktAoiSettings, userSettings)

//////////////////////////Test Coherence calculation on single scene and Batch of scenes.//////////////////////////////////

In [ ]:
%%time
# ###################This is for calculation of coherence##########################

# This is for testing one single list entry and diverse aoi with all possible overlapping szenarios
testElementSlice = ['/codede/Sentinel-1/SAR/SLC/2021/03/03/S1A_IW_SLC__1SDV_20210303T171640_20210303T171708_036837_0454E5_4937.SAFE',
  '/codede/Sentinel-1/SAR/SLC/2021/03/09/S1B_IW_SLC__1SDV_20210309T171611_20210309T171638_025941_031822_8848.SAFE, /codede/Sentinel-1/SAR/SLC/2021/03/09/S1B_IW_SLC__1SDV_20210309T171546_20210309T171613_025941_031822_F893.SAFE',
  'S1A_20210303_20210309_VVVH_15_desc_coh6d']

testElementSingle = ['/codede/Sentinel-1/SAR/SLC/2021/03/01/S1B_IW_SLC__1SDV_20210301T053314_20210301T053341_025817_03141D_C5C5.SAFE',
  '/codede/Sentinel-1/SAR/SLC/2021/03/07/S1A_IW_SLC__1SDV_20210307T053412_20210307T053439_036888_0456B1_5BAE.SAFE',
  'S1B_20210301_20210307_VVVH_66_asc_coh6d']

# Log file will be written and placed in folder in user settings main path
# Try both test elements and check for calculation times.
logOutput.createLogOutputFile("test_coherence")
specificSnapGraphProcessing.multiSceneProcCoherence(testElementSingle, "Test_AOI", wktAoiSettings, userSettings.cohOutputPath, logOutput)
logOutput.closeCurrentFile()

# This is for batch process of list
BatchProcessing(userSettings).calculateCoh(cohProcessingList6Days, cohProcessingList12Days, wktAoiSettings, userSettings)

In [ ]:
# Example for deriving Aoi of scenes
sceneList =['/codede/Sentinel-1/SAR/GRD/2020/01/11/S1B_IW_GRDH_1SDV_20200111T054051_20200111T054116_019765_0255EE_2BC5.SAFE', '/codede/Sentinel-1/SAR/GRD/2020/01/11/S1B_IW_GRDH_1SDV_20200111T054029_20200111T054054_019765_0255EE_6795.SAFE',
           '/codede/Sentinel-1/SAR/GRD/2020/01/16/S1B_IW_GRDH_1SDV_20200116T170030_20200116T170055_019845_025868_D776.SAFE', '/codede/Sentinel-1/SAR/GRD/2020/01/16/S1B_IW_GRDH_1SDV_20200116T170005_20200116T170030_019845_025868_218E.SAFE']
wktAoi = []
for i in sceneList:
    wktAoi.append(GeoPosition().getWktFromScene(i + "/manifest.safe"))
string = str(wktAoi)
print(string.replace("'", ''))

In [ ]:
# Example for performing graph sequence Split -> Slice -> Merge -> Subset on all SW of 2 scenes
scene1 = '/codede/Sentinel-1/SAR/SLC/2022/03/14/S1A_IW_SLC__1SDV_20220314T053418_20220314T053445_042313_050B38_C57E.SAFE'
scene2 = '/codede/Sentinel-1/SAR/SLC/2022/03/14/S1A_IW_SLC__1SDV_20220314T053442_20220314T053510_042313_050B38_0B46.SAFE'
specificSnapGraphProcessing.setSplitSliceMergeSubsetMultiPaths(scene1, scene2, userSettings.dataPath + "spacial_output/")

# ###################This performs Split Slice Merge Subset on two given scenes#########################
wktAoi="POLYGON((8.187561035156248 50.23315183247226,8.88519287109375 50.24369202255804,8.90167236328125 49.91939873027269,8.20404052734375 49.901711217260896,8.187561035156248 50.23315183247226))"
specificSnapGraphProcessing.setSubsetAoiValue(wktAoi)

specificSnapGraphProcessing.processSplitSliceMergeSubset(scene1)
specificSnapGraphProcessing.deleteAllTempFiles()

In [ ]:
%%time
# This is to performing S1 Slice-Assembly operator on a list of 2 scenes
scene1 = '/codede/Sentinel-1/SAR/GRD/2022/08/11/S1A_IW_GRDH_1SDV_20220811T173243_20220811T173308_044508_054FB6_45E2.SAFE'
scene2 = '/codede/Sentinel-1/SAR/GRD/2022/08/11/S1A_IW_GRDH_1SDV_20220811T173218_20220811T173243_044508_054FB6_EAB8.SAFE'

logOutput.createLogOutputFile("spacial_calc")
specificSnapGraphProcessing.setAndProcessSlice(scene1, scene2,  userSettings.dataPath + "spacial_output/", logOutput)
logOutput.closeCurrentFile()
specificSnapGraphProcessing.deleteAllTempFiles()